## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1a. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saint George,US,2021-11-06 17:08:31,37.1041,-113.5841,72.07,47,1,3.44,clear sky
1,1,Ushuaia,AR,2021-11-06 17:06:41,-54.8000,-68.3000,45.18,91,100,2.64,overcast clouds
2,2,Vaini,TO,2021-11-06 17:08:31,-21.2000,-175.2000,73.56,94,40,4.61,scattered clouds
3,3,Belaya Gora,RU,2021-11-06 17:08:32,68.5333,146.4167,11.08,94,95,12.82,light snow
4,4,Khatanga,RU,2021-11-06 17:08:32,71.9667,102.5000,11.32,93,100,13.80,overcast clouds


In [3]:
# 1b. Check counts to determine any NaN values
city_data_df.count()

City_ID                674
City                   674
Country                668
Date                   674
Lat                    674
Lng                    674
Max Temp               674
Humidity               674
Cloudiness             674
Wind Speed             674
Weather Description    674
dtype: int64

In [4]:
# 1c. Isolate rows with null values and crossreference the raw data to confirm the NA for the abbreviation of Namibia has been converted to NaN
# Used https://www.kite.com/python/answers/how-to-find-rows-with-nan-values-in-a-pandas-dataframe-in-python to help me isolate rows containing "NaN" values.

value_is_NaN = city_data_df.isnull()
row_has_NaN = value_is_NaN.any(axis=1)

print(city_data_df[row_has_NaN])

     City_ID          City Country                 Date      Lat      Lng  \
78        78        Tsumeb     NaN  2021-11-06 17:14:56 -19.2333  17.7167   
79        79     Maltahohe     NaN  2021-11-06 17:14:56 -24.8333  16.9833   
232      232         Opuwo     NaN  2021-11-06 17:19:57 -18.0607  13.8400   
261      261      Luderitz     NaN  2021-11-06 17:20:11 -26.6481  15.1594   
471      471    Walvis Bay     NaN  2021-11-06 17:26:36 -22.9575  14.5053   
546      546  Grootfontein     NaN  2021-11-06 17:28:05 -19.5667  18.1167   

     Max Temp  Humidity  Cloudiness  Wind Speed Weather Description  
78      89.76        22          99        9.22     overcast clouds  
79      79.92         9          54       19.62       broken clouds  
232     89.96        16         100       11.32     overcast clouds  
261     65.52        61         100       25.41     overcast clouds  
471     68.95        56          54       14.97       broken clouds  
546     81.97        34         100     

In [5]:
# 1d. Confirmed that Namibia's abbreviation was replaced with "NaN" and there were no other NaN values in the DataFrame
# 1e. Replace null values with 'NA'
city_data_df["Country"] = city_data_df["Country"].fillna('NA')

# 1f. Check to see if null values were replaced
city_data_df.loc[city_data_df["Country"] == 'NA'] 

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
78,78,Tsumeb,NA,2021-11-06 17:14:56,-19.2333,17.7167,89.76,22,99,9.22,overcast clouds
79,79,Maltahohe,NA,2021-11-06 17:14:56,-24.8333,16.9833,79.92,9,54,19.62,broken clouds
232,232,Opuwo,NA,2021-11-06 17:19:57,-18.0607,13.8400,89.96,16,100,11.32,overcast clouds
261,261,Luderitz,NA,2021-11-06 17:20:11,-26.6481,15.1594,65.52,61,100,25.41,overcast clouds
471,471,Walvis Bay,NA,2021-11-06 17:26:36,-22.9575,14.5053,68.95,56,54,14.97,broken clouds
546,546,Grootfontein,NA,2021-11-06 17:28:05,-19.5667,18.1167,81.97,34,100,3.44,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [7]:
# 3a. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saint George,US,2021-11-06 17:08:31,37.1041,-113.5841,72.07,47,1,3.44,clear sky
2,2,Vaini,TO,2021-11-06 17:08:31,-21.2000,-175.2000,73.56,94,40,4.61,scattered clouds
18,18,Bredasdorp,ZA,2021-11-06 17:08:37,-34.5322,20.0403,61.29,57,94,8.39,overcast clouds
19,19,San Quintin,MX,2021-11-06 17:08:37,30.4833,-115.9500,68.09,71,0,7.99,clear sky
20,20,Airai,TL,2021-11-06 17:08:38,-8.9266,125.4092,63.05,92,44,0.98,scattered clouds
27,27,Monopoli,IT,2021-11-06 17:08:41,40.9559,17.2896,65.71,95,100,10.18,overcast clouds
29,29,Marawi,PH,2021-11-06 17:08:42,7.9986,124.2928,69.66,95,99,0.67,overcast clouds
30,30,Port Elizabeth,ZA,2021-11-06 17:08:42,-33.9180,25.5701,64.09,76,100,3.89,overcast clouds
34,34,Cape Town,ZA,2021-11-06 17:05:43,-33.9258,18.4232,64.35,59,79,1.99,broken clouds
37,37,Muzaffarpur,IN,2021-11-06 17:08:44,26.1167,85.4000,66.13,53,37,4.45,scattered clouds


In [8]:
# 3b. Get a count of rows of the new DataFrame
len(preferred_cities_df)

180

In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                180
City                   180
Country                180
Date                   180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Weather Description    180
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Saint George,US,72.07,clear sky,37.1041,-113.5841,
2,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,
18,Bredasdorp,ZA,61.29,overcast clouds,-34.5322,20.0403,
19,San Quintin,MX,68.09,clear sky,30.4833,-115.9500,
20,Airai,TL,63.05,scattered clouds,-8.9266,125.4092,
27,Monopoli,IT,65.71,overcast clouds,40.9559,17.2896,
29,Marawi,PH,69.66,overcast clouds,7.9986,124.2928,
30,Port Elizabeth,ZA,64.09,overcast clouds,-33.9180,25.5701,
34,Cape Town,ZA,64.35,broken clouds,-33.9258,18.4232,
37,Muzaffarpur,IN,66.13,scattered clouds,26.1167,85.4000,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)

hotel_df.count()

City                   171
Country                171
Max Temp               171
Weather Description    171
Lat                    171
Lng                    171
Hotel Name             171
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))